### Parsing SLAC Fe55 Analysis files
- Craig Lage - 28-Oct-19

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys, glob, time, scipy
from scipy import stats
import pickle as pkl
from astropy.io import fits as pf

In [ ]:
# Now get Fe55 stamps
thedir = '/home/cslage/Research/LSST/code/poisson/fe55_data/'
E2VfitsFile = thedir+'R21_S10_6801D_psf_results_nsig4.fits'
E2VgainFile = thedir+'R21_S10_6801D_eotest_results.fits'
ITLfitsFile = thedir+'R20_S10_6801D_psf_results_nsig4.fits'
ITLgainFile = thedir+'R20_S10_6801D_eotest_results.fits'

In [ ]:
# Classes to put the data
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=np.linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=np.linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=np.zeros([nx,ny,nstamps])
        self.gain = 1.0


def BuildSpotList(fitsfilename, gainfilename, ampnumber, nx, ny):
    dy = 7; dx = 9 # Size of stamp in fitsfile catalog
    Kalpha = 1620 # Electrons in Kalpha line
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    hdulist=pf.open(fitsfilename)
    data=hdulist[ampnumber].data
    gain_hdulist=pf.open(gainfilename)
    gain = gain_hdulist[1].data['GAIN'][ampnumber-1]
    numRows = data.shape[0]
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numRows)
    spotlist.gain = gain
    mincounts = Kalpha * 0.95
    maxcounts = Kalpha * 1.05
    spotCounter = 0
    for row in range(numRows):
        prect_data = data[row]['PRECT_DATA']
        #print(row, prect_data.max())
        spot = np.zeros([nx,ny])
        for i in range(nx):
            for j in range(ny):
                spot[j,i] = prect_data[(i+1) + dx * (j+1)] * gain
        count = spot.sum()
        if count > mincounts and count < maxcounts:
            spotlist.data[:,:,spotCounter] = spot
            spotCounter += 1

    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,spotCounter)
    newspotlist.data = spotlist.data[:,:,0:spotCounter]
    newspotlist.gain = spotlist.gain
    del spotlist
    return newspotlist


In [ ]:
nx = 5; ny = 5; nAmps = 16
AveCounts = np.zeros([nx,ny])
TotalSpots = 0
for amp in range(nAmps):
    spotlist = BuildSpotList(E2VfitsFile, E2VgainFile, amp+1, nx, ny)
    AmpAve = spotlist.data.sum(axis=2) / float(spotlist.nstamps)
    AveCounts += AmpAve
    TotalSpots += spotlist.nstamps
    print(amp+1, spotlist.gain, spotlist.nstamps, AmpAve[2,2], AmpAve[2,3])

AveCounts /= float(nAmps)

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,1,1,aspect = 1)
plt.suptitle("Stacked Fe55 events - E2V, %d hits"%(TotalSpots), fontsize = 18)

for i in range(spotlist.nx):
    plt.plot([spotlist.x[i]-0.5, spotlist.x[i]-0.5], [spotlist.y[0]-0.5, spotlist.y[-1]+0.5], color = 'black', ls = '--')
    plt.plot([spotlist.x[i]+0.5, spotlist.x[i]+0.5], [spotlist.y[0]-0.5, spotlist.y[-1]+0.5], color = 'black', ls = '--')
for j in range(spotlist.ny):
    plt.plot([spotlist.x[0]-0.5, spotlist.x[-1]+0.5], [spotlist.y[j]-0.5, spotlist.y[j]-0.5], color = 'black', ls = '--')
    plt.plot([spotlist.x[0]-0.5, spotlist.x[-1]+0.5], [spotlist.y[j]+0.5, spotlist.y[j]+0.5], color = 'black', ls = '--')


for i in range(spotlist.nx):
    for j in range(spotlist.ny):
        plt.text(spotlist.x[i] - 0.30, spotlist.y[j] + 0.15, "%.1f"%AveCounts[i,j], color = 'green')

print(AveCounts.sum())
plt.savefig(thedir+'E2V_Fe55_Stacked_28Oct19.pdf')

In [ ]:
nx = 5; ny = 5; nAmps = 11
AveCounts = np.zeros([nx,ny])
TotalSpots = 0
for amp in range(nAmps):
    spotlist = BuildSpotList(ITLfitsFile, ITLgainFile, amp+1, nx, ny)
    AmpAve = spotlist.data.sum(axis=2) / float(spotlist.nstamps)
    AveCounts += AmpAve
    TotalSpots += spotlist.nstamps
    print(amp+1, spotlist.gain, spotlist.nstamps, AmpAve[2,2], AmpAve[2,3])
AveCounts /= float(nAmps)

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,1,1,aspect = 1)
plt.suptitle("Stacked Fe55 events - ITL, %d hits"%(TotalSpots), fontsize = 18)

for i in range(spotlist.nx):
    plt.plot([spotlist.x[i]-0.5, spotlist.x[i]-0.5], [spotlist.y[0]-0.5, spotlist.y[-1]+0.5], color = 'black', ls = '--')
    plt.plot([spotlist.x[i]+0.5, spotlist.x[i]+0.5], [spotlist.y[0]-0.5, spotlist.y[-1]+0.5], color = 'black', ls = '--')
for j in range(spotlist.ny):
    plt.plot([spotlist.x[0]-0.5, spotlist.x[-1]+0.5], [spotlist.y[j]-0.5, spotlist.y[j]-0.5], color = 'black', ls = '--')
    plt.plot([spotlist.x[0]-0.5, spotlist.x[-1]+0.5], [spotlist.y[j]+0.5, spotlist.y[j]+0.5], color = 'black', ls = '--')


for i in range(spotlist.nx):
    for j in range(spotlist.ny):
        plt.text(spotlist.x[i] - 0.30, spotlist.y[j] + 0.15, "%.1f"%AveCounts[i,j], color = 'green')



print(AveCounts.sum())
plt.savefig(thedir+'ITL_Fe55_Stacked_28Oct19.pdf')